In [1]:
import pdfplumber
from typing import Iterator
from langchain_core.documents import Document
from tqdm import tqdm
from pprint import pprint
import re
import os

## 파일 읽기

In [2]:
def has_subfolders(directory) -> bool:
    try:
        items = os.listdir(directory)
        for item in items:
            if os.path.isdir(os.path.join(directory, item)):
                return True
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

In [3]:
total_results =[]
def get_folders_files(path:str) -> list:   # 폴더 트리를 리커시브하게 읽어서 전체 PDF 파일 full 경로 수집 
    global total_results
    
    all_items = os.listdir(path)
    files = [f for f in all_items if os.path.isfile(os.path.join(path, f))]
    results = [os.path.join(path, file) for file in files]
    total_results.extend(results)

    dirs = [f for f in all_items if os.path.isdir(os.path.join(path, f))]
    if dirs:
        dirs = [path+"\\" + lv2_dir for lv2_dir in dirs]
        for dir in dirs:
            get_folders_files(dir)
    return total_results

In [4]:
lv1_dir = ".\\2024"
get_folders_files(lv1_dir)

['.\\2024\\Manual\\Guidance for Autonomous Ships_2023.pdf',
 '.\\2024\\POS\\Air Compressor.pdf',
 '.\\2024\\POS\\FWG.pdf',
 '.\\2024\\POS\\path_to_save_image_2.pdf',
 '.\\2024\\POS\\temp\\Observations on Building RAG System for Technical Documents.pdf',
 '.\\2024\\Rule\\PART 4_2023_hull_equipment.pdf']

In [5]:
# current_directory = os.getcwd()
# print("Current Directory:", current_directory)
# folder_name = "2024"
# folder_path = os.path.join(current_directory, folder_name)
# print("Folder Path:", folder_path)
# all_items = os.listdir(folder_path)  # 하위 폴더까지 읽기
# print(all_items)
# files = [f for f in all_items if os.path.isfile(os.path.join(folder_path, f))]  # 폴더 제외 파일만 읽기
# print(files)  

# Basic

In [5]:
# 샘플 검토 대상 문서 지정 ---> full_path
# current_directory = os.getcwd()
# print("Current Directory:", current_directory)
# folder_name = "files"
# folder_path = os.path.join(current_directory, folder_name)
# print("Folder Path:", folder_path)
# # all_items = os.listdir(folder_path)  # 하위 폴더까지 읽기
# # print(all_items)
# # files = [f for f in all_items if os.path.isfile(os.path.join(folder_path, f))]  # 폴더 제외 파일만 읽기
# # print(files)  
# # target_file = files[0]
# # print(target_file)
# folder_path = os.path.join(current_directory, folder_name)
# full_path = os.path.join(folder_path, target_file)
# print(full_path)

Current Directory: d:\AA_develop\parsing
Folder Path: d:\AA_develop\parsing\files


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'd:\\AA_develop\\parsing\\files'

In [7]:
def create_folder_if_not_exists(folder_path:str):  # 이미지 저장 폴더 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"폴더가 생성되었습니다: {folder_path}")
    else:
        print(f"폴더가 이미 존재합니다: {folder_path}")

def save_pdf_to_img(path:str, file_name:str, page_num:int):  # pdf를 png 이미지 파일로 저장
    with pdfplumber.open(path) as pdf:
        # print(len(pdf.pages))
        page = pdf.pages[page_num]
        # print(page)
        im = page.to_image(resolution=150)
        # im.draw_rects(first_page.extract_words())  # 글자에 Red Box 그리기
        save_path = f"{os.getcwd()}/images/{file_name}/{file_name}_{page_num}.png"
        # print(f"save_path: {save_path}")
        im.save(save_path, format="PNG", )
        # im.show()

In [8]:
def text_parser(path:str, page_num:int, crop:bool):   # 텍스트 파싱, A4상단 표준 크롭핑 적용 선택 가능
    with pdfplumber.open(path) as pdf:
        page = pdf.pages[page_num]
        if crop:
            bounding_box = (3, 70, 590, 770)   #default : (0, 0, 595, 841)
            page = page.crop(bounding_box, relative=False, strict=True)
        else: pass
        result = page.extract_text(layout=False)
    return result

In [9]:
text_parser(total_results[0], 2, True)

'CONTENTS\nCHAPTER 1 GENERAL···········································································································1\nSection 1 General············································································································1\nSection 2 Operation Plan·······························································································4\nSection 3 Cyber security································································································5\nCHAPTER 2 AUTONOMOUS SYSTEMS AND AUTONOMOUS SHIPS·························7\nSection 1 Configuration and Function of Autonomous Systems···························7\nSection 2 Requirements for Autonomous Systems and Autonomous Ships···14\nSection 3 Approval procedure of autonomous ships·············································19\nCHAPTER 3 RISK-BASED APPROVAL ···········································································21\nSection 1 General····························

In [10]:
# full_path = "D:/AA_develop/parsing/2024/POS/FWG.pdf"
# with pdfplumber.open(full_path) as pdf:
#     first_page = pdf.pages[5]
#     print(first_page)
#     result = first_page.search("The material of insulation, gasket and/or gland packing shall be", regex=True, case=False, layout=True)
#     print(result)

<Page:6>
[{'text': 'The material of insulation, gasket and/or gland packing shall be', 'x0': 77.904, 'top': 115.93999999999994, 'x1': 413.37799999999993, 'bottom': 127.93999999999994, 'groups': (), 'chars': [{'matrix': (1, 0, 0, 1, 77.904, 716.62), 'fontname': 'ArialMT', 'adv': 7.332, 'upright': True, 'x0': 77.904, 'y0': 714.1, 'x1': 85.23599999999999, 'y1': 726.1, 'width': 7.331999999999994, 'height': 12.0, 'size': 12.0, 'mcid': 2, 'tag': 'P', 'object_type': 'char', 'page_number': 6, 'ncs': 'DeviceGray', 'text': 'T', 'stroking_color': (0,), 'stroking_pattern': None, 'non_stroking_color': (0,), 'non_stroking_pattern': None, 'top': 115.93999999999994, 'bottom': 127.93999999999994, 'doctop': 4325.9}, {'matrix': (1, 0, 0, 1, 85.344, 716.62), 'fontname': 'ArialMT', 'adv': 6.672000000000001, 'upright': True, 'x0': 85.344, 'y0': 714.1, 'x1': 92.01599999999999, 'y1': 726.1, 'width': 6.671999999999997, 'height': 12.0, 'size': 12.0, 'mcid': 2, 'tag': 'P', 'object_type': 'char', 'page_number': 6

In [11]:
table_settings={
    "vertical_strategy": "lines", 
    "horizontal_strategy": "lines",
    "explicit_vertical_lines": [],
    "explicit_horizontal_lines": [],
    "snap_tolerance": 3,
    "snap_x_tolerance": 3,
    "snap_y_tolerance": 3,
    "join_tolerance": 3,
    "join_x_tolerance": 3,
    "join_y_tolerance": 3,
    "edge_min_length": 3,
    "min_words_vertical": 3,
    "min_words_horizontal": 1,
    "intersection_tolerance": 3,
    "intersection_x_tolerance": 3,
    "intersection_y_tolerance": 3,
    "text_tolerance": 3,
    "text_x_tolerance": 3,
    "text_y_tolerance": 3,
    # "text_*": …, # See below
    }

In [12]:
def convert_header_to_separator(header: str) -> str:
    """
    Convert a markdown table header row into a separator row.

    Args:
    header (str): The header row string.

    Returns:
    str: The separator row string.
    """
    # Use a regex to replace each header content with the appropriate number of hyphens
    separator = re.sub(r'[^|]+', lambda m: '-' * len(m.group(0)), header)
    return separator

In [13]:
def table_parser(pdf_path:str, page_num:int, crop:bool):   # 테이블 파싱(마크다운 형식), A4상단 표준 크롭핑 적용 선택 가능
    with pdfplumber.open(pdf_path) as pdf:
        # Find the examined page
        table_page = pdf.pages[page_num]
        if crop:
            bounding_box = (3, 70, 590, 770)   #default : (0, 0, 595, 841)
            table_page = table_page.crop(bounding_box, relative=False, strict=True)
        else: pass
        tables = table_page.extract_tables(table_settings = table_settings)
        if tables:
            for table in tables:
                table_string = ''
                # Iterate through each row of the table
                for row_num in range(len(table)):
                    row = table[row_num]
                    # Remove the line breaker from the wrapped texts
                    cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
                    # Convert the table into a string 
                    table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
                    if row_num ==0:  # 첫줄 작업이면, Header Line 추가
                        header_line = convert_header_to_separator(table_string[:-1])
                        table_string+= header_line+'\n'
                # Removing the last line break
                table_string = table_string[:-1]

        return table_string

In [23]:
pprint(table_parser(total_results[2], 8, False))

('|Query ID|Query|Expected answer|Generated response – similarity by '
 'paragraph||Generated Response – similarity by sentence, '
 'retrieve||Observations|\n'
 '|--------|-----|---------------|--------------------------------------------||-----------------------------------------------------||------------|\n'
 '|None|None|None|None|None|paragraph|None|None|\n'
 '|F1|What do the values of RAW Group Indication subfield in RPS element '
 'indicate?|The RAW Group Indication subfield indicates whether the RAW Group '
 'subfield is present in the RAW Assignment subfield and is interpreted as '
 'follows: — When the RAW type is generic RAW, sounding RAW, or triggering '
 'frame RAW, the RAW Group Indication subfield indicates whether the RAW group '
 'defined in the current RAW assignment is the same RAW group as defined in '
 'the previous RAW assignment. When the RAW Group Indication subfield is equal '
 'to 0, the RAW group defined in the current RAW assignment is the same as the '
 'RAW 

In [15]:
def extract_level_name(path:str) -> list: 
    temp = path.split("\\")
    lv1 = temp[1]
    if temp[2]:
        if temp[2] != temp[-1]:
            lv2 = temp[2]
            lv3 = temp[-1].replace(".pdf", "")
        else:
            lv2 = None
            lv3 = temp[-1].replace(".pdf", "")
    result = [lv1, lv2, lv3]
    return result

extract_level_name(total_results[0])

['2024', 'Manual', 'Guidance for Autonomous Ships_2023']

In [16]:
def main_document_converter(path:str, crop:bool) -> Iterator[Document]:  # <-- Does not take any arguments
    full_result = []
    file_name = path.split("\\")[-1].split(".")[0].strip() # for metadata
    img_save_folder = os.path.join(os.getcwd(), f"images/{file_name}")  # images 폴더 생성후 그 안에 file_name폴더 생성
    create_folder_if_not_exists(img_save_folder)

    with pdfplumber.open(path) as pdf:
        page_number = 0  # for metadata
        for _ in tqdm(pdf.pages):
            
            save_pdf_to_img(path, file_name, page_number) # pdf_to_png 
            text_result = text_parser(path, page_number, crop)  # for page_content
            table_result = table_parser(path, page_number, crop)  # for page_content
            level_names = extract_level_name(path)

            if table_result:
                total_page_result = text_result + "\n\n" + table_result   # table_result가 있으면, text_result 끝에 엔터후 이어붙이기
                result = Document(
                    page_content=total_page_result,
                    metadata={"page_number": page_number, "lv1":level_names[0], "lv2": level_names[1], "lv3": level_names[2], "source": path},
                    )
            else:
                result = Document(
                    page_content=text_result,
                    metadata={"page_number": page_number, "lv1":level_names[0], "lv2": level_names[1], "lv3": level_names[2], "source": path},
                    )
            full_result.append(result)
            page_number += 1
        parsed_document = full_result
    return parsed_document   # langchain Document type

In [17]:
total_results

['.\\2024\\Manual\\Guidance for Autonomous Ships_2023.pdf',
 '.\\2024\\POS\\FWG.pdf',
 '.\\2024\\POS\\temp\\Observations on Building RAG System for Technical Documents.pdf',
 '.\\2024\\Rule\\PART 4_2023_hull_equipment.pdf']

In [18]:
try: 
    print("Do Cropping")
    result = main_document_converter(path=total_results[2], crop=True)
except:
    print("Do Not Cropping")
    result = main_document_converter(path=total_results[2], crop=False)

print(type(result), type(result[0]))
pprint(result)

Do Cropping
폴더가 이미 존재합니다: d:\AA_develop\parsing\images/Observations on Building RAG System for Technical Documents


  0%|          | 0/9 [00:00<?, ?it/s]

 44%|████▍     | 4/9 [00:01<00:02,  2.42it/s]


Do Not Cropping
폴더가 이미 존재합니다: d:\AA_develop\parsing\images/Observations on Building RAG System for Technical Documents


100%|██████████| 9/9 [00:05<00:00,  1.71it/s]

<class 'list'> <class 'langchain_core.documents.base.Document'>
[Document(metadata={'page_number': 0, 'lv1': '2024', 'lv2': 'POS', 'lv3': 'Observations on Building RAG System for Technical Documents', 'source': '.\\2024\\POS\\temp\\Observations on Building RAG System for Technical Documents.pdf'}, page_content='PublishedasaTinyPaperatICLR2024\nOBSERVATIONS ON BUILDING RAG SYSTEMS FOR\nTECHNICAL DOCUMENTS\nSumitSomanandSujoyRoychowdhury∗\n{sumit.soman, sujoy.roychowdhury}@ericsson.com\nABSTRACT\nRetrieval augmented generation (RAG) for technical documents creates chal-\nlengesasembeddingsdonotoftencapturedomaininformation. Wereviewprior\nartforimportantfactorsaffectingRAGandperformexperimentstohighlightbest\npracticesandpotentialchallengestobuildRAGsystemsfortechnicaldocuments.\n1 INTRODUCTION\nLongformQuestionAnswering(QA)involvesgeneratingparagraph-sizeresponsesfromLargeLan-\nguageModels(LLMs). RAGfortechnicaldocumentshasseveralchallengesXuetal.(2023);Toro\net al. (2023). Factors affe

In [19]:
pprint(result[8].page_content)

('Table 3 – Generator Performance\n'
 'Query ID Query Expected answer Generated response – similarity by paragraph '
 'Generated Response – similarity by sentence, retrieve Observations\n'
 'paragraph\n'
 "F1 What do the values The RAW Group Indication subfield indicates 'According "
 'to the given paragraphs, the values of According to the given paragraphs, '
 'the values of the RAW A much broader context in\n'
 'of RAW Group whether the RAW Group subfield is present in the RAW Group '
 'Indication subfield in the RPS Group Indication subfield in the RPS (Radio '
 'Path Selection) general being available for\n'
 'Indication subfield in the RAW (Radio Parameter Set) element indicate '
 'whether element indicate whether the RAW Group subfield is the LLM because '
 'of multiple\n'
 'RPS element Assignment subfield and is interpreted as the RAW Group subfield '
 'is present or not in the present in the RAW Assignment subfield. If the '
 'value of retrieved paragraphs helps it\n'
 'indicat

In [48]:
total_results

['.\\2024\\Manual\\Guidance for Autonomous Ships_2023.pdf',
 '.\\2024\\POS\\FWG.pdf',
 '.\\2024\\POS\\temp\\Observations on Building RAG System for Technical Documents.pdf',
 '.\\2024\\Rule\\PART 4_2023_hull_equipment.pdf']

In [49]:
pdf_path = total_results[3]
pdf_path

'.\\2024\\Rule\\PART 4_2023_hull_equipment.pdf'

In [50]:
def get_table_bbox(path:str, page_num:int) -> list:
    result = []
    with pdfplumber.open(path) as pdf:
        pages = pdf.pages
        page = pages[page_num]
        for table in page.find_tables():
            bbox = table.bbox
            result.append(bbox)
    return result

In [51]:
get_table_bbox(pdf_path, 6)

[(119.696, 328.44100000000003, 460.912, 454.782),
 (129.53, 495.897, 451.197, 585.2)]

In [54]:
def text_parser(path:str, page_num:int, crop:bool):   # 텍스트 파싱, A4상단 표준 크롭핑 적용 선택 
    with pdfplumber.open(path) as pdf:
        page = pdf.pages[page_num]
        if crop:
            bounding_box = (3, 70, 590, 770)   #default : (0, 0, 595, 841)
            page = page.crop(bounding_box, relative=False, strict=True)
        else: pass

        bbox_list = get_table_bbox(path=path, page_num=page_num)
        print(bbox_list)

        

        if bbox_list:
            for bbox in bbox_list:
                text_above_table = page.within_bbox((0, 0, page.width, bbox[1])).extract_text()
                text_below_table = page.within_bbox((0, bbox[3], page.width, page.height)).extract_text()

        page.ta



    #     result = page.extract_text(layout=False)
    # return result

In [55]:
pprint(text_parser(pdf_path, 6, True))

[(119.696, 328.44100000000003, 460.912, 454.782), (129.53, 495.897, 451.197, 585.2)]
None


In [57]:
def table_parser(pdf_path:str, page_num:int, crop:bool):   # 테이블 파싱(마크다운 형식), A4상단 표준 크롭핑 적용 선택 가능
    full_table = []
    with pdfplumber.open(pdf_path) as pdf:
        # Find the examined page
        table_page = pdf.pages[page_num]
        if crop:
            bounding_box = (3, 70, 590, 770)   #default : (0, 0, 595, 841)
            table_page = table_page.crop(bounding_box, relative=False, strict=True)
        else: pass
        tables = table_page.extract_tables(table_settings = table_settings)
        if tables:
            for table in tables:
                table_string = ''
                # Iterate through each row of the table
                for row_num in range(len(table)):
                    row = table[row_num]
                    # Remove the line breaker from the wrapped texts
                    cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
                    # Convert the table into a string 
                    table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
                    if row_num ==0:  # 첫줄 작업이면, Header Line 추가
                        header_line = convert_header_to_separator(table_string[:-1])
                        table_string+= header_line+'\n'
                # Removing the last line break
                table_string = table_string[:-1]
                full_table.append(table_string)

        return full_table

In [58]:
table_parser(pdf_path, 6, True)

['|\ue0ae (N/mm2) \ue0fd|\ue00a|\n|-----------|-|\n|\ue0ae \ue056\ue035\ue036\ue038 \ue0fd|\ue044\ue035\ue036\ue038\ue045\ue03d\ue053\ue03a\ue038 \ue00a\ue047 \ue06d \ue0ae \ue0fd|\n|\ue0ae ≤\ue035\ue036\ue038 \ue0fd|\ue044\ue035\ue036\ue038\ue045\ue034\ue053\ue03d \ue00a\ue047 \ue06d \ue0ae \ue0fd|\n|\ue0ae = yield stress(N/mm2) of material used, and is not to be taken greater \ue0fd than 0.7\ue0ae or 450(N/mm2), whichever is smaller value. \ue013 \ue0ae = specified minimum tensile strength of material used(N/mm2). \ue013|None|',
 '|Material|\ue00a|\n|--------|-|\n|A, B, D or E|1.0|\n|AH32, DH32 or EH32|0.78|\n|AH36, DH36 or EH36|0.72|\n|AH40, DH40 or EH40|0.68|']

In [ ]:
import pdfplumber

# Path to the PDF file
pdf_path = "your_pdf_file.pdf"

# Open the PDF
with pdfplumber.open(pdf_path) as pdf:
    # Select the page you want to parse
    page = pdf.pages[0]  # Replace 0 with the appropriate page number

    # Define the bounding boxes of the two tables
    # (left, top, right, bottom) for each table
    table1_bbox = (100, 200, 500, 300)  # Example coordinates for the first table
    table2_bbox = (100, 500, 500, 600)  # Example coordinates for the second table

    # Get the text from the area above the first table
    text_above_table1 = page.within_bbox((0, 0, page.width, table1_bbox[1])).extract_text()

    # Get the text from the area between the first and second table
    text_between_tables = page.within_bbox((0, table1_bbox[3], page.width, table2_bbox[1])).extract_text()

    # Get the text from the area below the second table
    text_below_table2 = page.within_bbox((0, table2_bbox[3], page.width, page.height)).extract_text()

    # Combine the extracted text
    full_text = (
        (text_above_table1 or "") + "\n" +
        (text_between_tables or "") + "\n" +
        (text_below_table2 or "")
    )

    print(full_text)


In [71]:
path = "D:/AA_develop/parsing/2024/Manual/BERT.pdf"
# path = "D:/AA_develop/parsing/2024/POS/FWG.pdf"
with pdfplumber.open(path) as pdf:
    page = pdf.pages[0]
    print(page)
    texts = page.extract_text(layout=False)
    print(texts)

<Page:1>
BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding
JacobDevlin Ming-WeiChang KentonLee KristinaToutanova
GoogleAILanguage
{jacobdevlin,mingweichang,kentonl,kristout}@google.com
Abstract There are two existing strategies for apply-
ingpre-trainedlanguage representations todown-
We introduce a new language representa-
stream tasks: feature-based and fine-tuning. The
tion model called BERT, which stands for
feature-based approach, such as ELMo (Peters
Bidirectional Encoder Representations from
Transformers. Unlike recent language repre- et al., 2018a), uses task-specific architectures that
sentation models (Peters et al., 2018a; Rad- include the pre-trained representations as addi-
ford et al., 2018), BERT is designed to pre- tional features. The fine-tuning approach, such as
train deep bidirectional representations from the Generative Pre-trained Transformer (OpenAI
unlabeledtextbyjointlyconditioningonboth
GPT) (Radford et al., 2018), introduces min

In [74]:
with pdfplumber.open(path) as pdf:
    page = pdf.pages[0]
    # Get the width of the page to divide it into two columns
    width = page.width

    # Define the bounding box for the left column (x0, top, x1, bottom)
    left_bbox = (0, 0, width / 2, page.height)

    # Define the bounding box for the right column
    right_bbox = (width / 2, 0, width, page.height)

    # Extract text from the left column
    left_text = page.within_bbox(left_bbox).extract_text()

    # Extract text from the right column
    right_text = page.within_bbox(right_bbox).extract_text()

    # Print the extracted text
    print("Left Column:\n", left_text)
    print("\nRight Column:\n", right_text)

Left Column:
 BERT: Pre-training of Deep
Language U
JacobDevlin Ming-WeiChang
GoogleA
{jacobdevlin,mingweichang,
Abstract
We introduce a new language representa-
tion model called BERT, which stands for
Bidirectional Encoder Representations from
Transformers. Unlike recent language repre-
sentation models (Peters et al., 2018a; Rad-
ford et al., 2018), BERT is designed to pre-
train deep bidirectional representations from
unlabeledtextbyjointlyconditioningonboth
left and right context in all layers. As a re-
sult,thepre-trainedBERTmodelcanbefine-
tuned with just one additional output layer
to create state-of-the-art models for a wide
rangeoftasks,suchasquestionansweringand
language inference, without substantial task-
specificarchitecturemodifications.
BERT is conceptually simple and empirically
powerful. It obtains new state-of-the-art re-
sults on eleven natural language processing
tasks, including pushing the GLUE score to
80.5% (7.7% point absolute improvement),
MultiNLI accuracy t

In [67]:
from collections import Counter

def detect_two_column_layout(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            words = page.extract_words()
            x_coords = [word['x0'] for word in words]
        

            # Cluster x-coordinates into groups (bins)
            bins = Counter()
            for x in x_coords:
                # Group coordinates into bins of width 50 (adjustable)
                bin_key = int(x // 2) * 2                
                bins[bin_key] += 1

            # Determine the number of distinct clusters (bins with many words)
            cluster_threshold = 50  # This is adjustable
            clusters = [count for count in bins.values() if count > cluster_threshold]

            if len(clusters) > 1:
                print(f"Page {page_number}: Detected a multi-column layout {len(clusters)}.")
            else:
                print(f"Page {page_number}: Single-column layout detected. {len(clusters)}")

In [69]:
path = "D:/AA_develop/parsing/2024/Manual/BERT.pdf"
# path = "D:/AA_develop/parsing/2024/POS/FWG.pdf"

detect_two_column_layout(path)

Page 1: Single-column layout detected. 0
Page 2: Single-column layout detected. 0
Page 3: Single-column layout detected. 0
Page 4: Single-column layout detected. 0
Page 5: Single-column layout detected. 0
Page 6: Single-column layout detected. 0
Page 7: Single-column layout detected. 0
Page 8: Single-column layout detected. 0
Page 9: Single-column layout detected. 0
Page 10: Single-column layout detected. 0
Page 11: Single-column layout detected. 0
Page 12: Single-column layout detected. 0
Page 13: Single-column layout detected. 0
Page 14: Single-column layout detected. 0
Page 15: Single-column layout detected. 0
Page 16: Single-column layout detected. 0


In [13]:
import re
header = "||HEATING MEDIUM|COOLING MEDIUM|"
# separator = re.sub(r'[^|]+', lambda m: '-' * 3, header)
separator = re.sub(r'[^|]+', lambda m: '-' * len(m.group(0)), header)
separator = separator.replace("||", "|-|", 1)

print(separator)

|-|--------------|--------------|


In [9]:
for i in lambda m: '-' * len(m.group(0)):
    print(i)
    break

TypeError: 'function' object is not iterable